In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import random

import torchvision.transforms as transforms
import torchvision.datasets as dset
# from scipy.misc import imread
from imageio import imread

from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
# from model.nms.nms_wrapper import nms
from model.roi_layers import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
import pdb
import base64
import matplotlib.pyplot as plt

In [20]:
def parse_args():
  """
  Parse input arguments
  """
  parser = argparse.ArgumentParser(description='Train a Fast R-CNN network')
  parser.add_argument('--dataset', dest='dataset',
                      help='training dataset',
                      default='vg', type=str)
  parser.add_argument('--cfg', dest='cfg_file',
                      help='optional config file',
                      default='cfgs/res101.yml', type=str)
  parser.add_argument('--net', dest='net',
                      help='vgg16, res50, res101, res152',
                      default='res101', type=str)
  parser.add_argument('--load_dir', dest='load_dir',
                      help='directory to load models',
                      default="models")
  parser.add_argument('--image_dir', dest='image_dir',
                      help='directory to load images for demo',
                      default="images")
  parser.add_argument('--image_file', dest='image_file',
                      help='the file name of load images for demo',
                      default="img1.jpg")
  parser.add_argument('--classes_dir', dest='classes_dir',
                      help='directory to load object classes for classification',
                      default="data/genome/1600-400-20")
  parser.add_argument('--cuda', dest='cuda',
                      help='whether use CUDA',
                      action='store_true')
  parser.add_argument('--mGPUs', dest='mGPUs',
                      help='whether use multiple GPUs',
                      action='store_true')
  parser.add_argument('--set', dest='set_cfgs',
                      help='set config keys', default=None,
                      nargs=argparse.REMAINDER)
  parser.add_argument('--cag', dest='class_agnostic',
                      help='whether perform class_agnostic bbox regression',
                      action='store_true')
  parser.add_argument('--parallel_type', dest='parallel_type',
                      help='which part of model to parallel, 0: all, 1: model before roi pooling',
                      default=0, type=int)
  parser.add_argument('--vis', dest='vis',
                      help='visualization mode',
                      action='store_true')

  args = parser.parse_args()
  return args

lr = cfg.TRAIN.LEARNING_RATE
momentum = cfg.TRAIN.MOMENTUM
weight_decay = cfg.TRAIN.WEIGHT_DECAY


conf_thresh = 0.4
MIN_BOXES = 10
MAX_BOXES = 36

In [21]:
def _get_image_blob(im):
  """Converts an image into a network input.
  Arguments:
    im (ndarray): a color image in BGR order
  Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
  """
  im_orig = im.astype(np.float32, copy=True)
  im_orig -= cfg.PIXEL_MEANS

  im_shape = im_orig.shape
  im_size_min = np.min(im_shape[0:2])
  im_size_max = np.max(im_shape[0:2])

  processed_ims = []
  im_scale_factors = []

  for target_size in cfg.TEST.SCALES:
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
      im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
    im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,
            interpolation=cv2.INTER_LINEAR)
    im_scale_factors.append(im_scale)
    processed_ims.append(im)

  # Create a blob to hold the input images
  blob = im_list_to_blob(processed_ims)

  return blob, np.array(im_scale_factors)


In [22]:
def load_model():
    
    #set cfg_file
    cfg_file = "cfgs/res101.yml"
    # set cfg according to the dataset used to train the pre-trained model
    dataset_set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']

    cfg_from_file(cfg_file)
    cfg_from_list(dataset_set_cfgs)

    cfg.USE_GPU_NMS = True

    print('Using config:')
    pprint.pprint(cfg)
    np.random.seed(cfg.RNG_SEED)

    # Load classes
    classes = ['__background__']
    with open('./dataset/objects_vocab.txt') as f:
        for object in f.readlines():
            classes.append(object.split(',')[0].lower().strip())

    load_name = 'faster_rcnn_res101_vg.pth'

    # initilize the network here. the network used to train the pre-trained model

    fasterRCNN = resnet(classes, 101, pretrained=True, class_agnostic=True)
    

    fasterRCNN.create_architecture()
    print(fasterRCNN)
    
    checkpoint = torch.load(load_name, map_location=(lambda storage, loc: storage))
    try:
      fasterRCNN.load_state_dict(checkpoint['model'])
    except RuntimeError as e:
      print('Ignoring "' + str(e) + '"')

    if 'pooling_mode' in checkpoint.keys():
      cfg.POOLING_MODE = checkpoint['pooling_mode']

    print('load model successfully!')

    print("load model %s" % (load_name))

    return classes, fasterRCNN

In [23]:
def get_detections_from_im(fasterRCNN, classes, im_file, conf_thresh=0.5):
    """obtain the image_info for each image,
    im_file: the path of the image

    return: dict of { 'image_h', 'image_w', 'num_boxes', 'boxes', 'features'}
    boxes: the coordinate of each box
    """
    # initilize the tensor holder here.
    im_data = torch.FloatTensor(1)
    im_info = torch.FloatTensor(1)
    num_boxes = torch.LongTensor(1)
    gt_boxes = torch.FloatTensor(1)

    # make variable
    with torch.no_grad():
        im_data = Variable(im_data)
        im_info = Variable(im_info)
        num_boxes = Variable(num_boxes)
        gt_boxes = Variable(gt_boxes)


    fasterRCNN.eval()
    max_per_image = 100
    thresh = 0.07

    #load images
    # im = cv2.imread(im_file)
    im_in = np.array(imread(im_file))
    if len(im_in.shape) == 2:
      im_in = im_in[:,:,np.newaxis]
      im_in = np.concatenate((im_in,im_in,im_in), axis=2)
    # rgb -> bgr
    im = im_in[:,:,::-1]

    vis = True

    blobs, im_scales = _get_image_blob(im)
    assert len(im_scales) == 1, "Only single-image batch implemented"
    im_blob = blobs
    im_info_np = np.array([[im_blob.shape[1], im_blob.shape[2], im_scales[0]]], dtype=np.float32)

    im_data_pt = torch.from_numpy(im_blob)
    im_data_pt = im_data_pt.permute(0, 3, 1, 2)
    im_info_pt = torch.from_numpy(im_info_np)

    with torch.no_grad():
            im_data.resize_(im_data_pt.size()).copy_(im_data_pt)
            im_info.resize_(im_info_pt.size()).copy_(im_info_pt)
            gt_boxes.resize_(1, 1, 5).zero_()
            num_boxes.resize_(1).zero_()
    # pdb.set_trace()
    det_tic = time.time()

    # the region features[box_num * 2048] are required.
    rois, cls_prob, bbox_pred, \
    rpn_loss_cls, rpn_loss_box, \
    RCNN_loss_cls, RCNN_loss_bbox, \
    rois_label, pooled_feat = fasterRCNN(im_data, im_info, gt_boxes, num_boxes, pool_feat = True)

    scores = cls_prob.data
    boxes = rois.data[:, :, 1:5]

    if cfg.TEST.BBOX_REG:
        # Apply bounding-box regression deltas
        box_deltas = bbox_pred.data
        if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
        # Optionally normalize targets by a precomputed mean and stdev
            box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS) \
                             + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS)
            box_deltas = box_deltas.view(1, -1, 4)

        pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
        pred_boxes = clip_boxes(pred_boxes, im_info.data, 1)
    else:
        # Simply repeat the boxes, once for each class
        pred_boxes = np.tile(boxes, (1, scores.shape[1]))

    pred_boxes /= im_scales[0]

    scores = scores.squeeze()
    pred_boxes = pred_boxes.squeeze()

    det_toc = time.time()
    detect_time = det_toc - det_tic
    misc_tic = time.time()

    max_conf = torch.zeros((pred_boxes.shape[0]))

    if vis:
        im2show = np.copy(im)
    for j in range(1, len(classes)):
        inds = torch.nonzero(scores[:,j]>conf_thresh).view(-1)
        # if there is det
        if inds.numel() > 0:
          cls_scores = scores[:,j][inds]
          _, order = torch.sort(cls_scores, 0, True)
          cls_boxes = pred_boxes[inds, :]


          cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
          # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
          cls_dets = cls_dets[order]
          # keep = nms(cls_dets, cfg.TEST.NMS, force_cpu=not cfg.USE_GPU_NMS)
          keep = nms(cls_boxes[order, :], cls_scores[order], cfg.TEST.NMS)
          cls_dets = cls_dets[keep.view(-1).long()]
          index = inds[order[keep]]
          max_conf[index] = torch.where(scores[index, j] > max_conf[index], scores[index, j], max_conf[index])
          if vis:
            im2show = vis_detections(im2show, classes[j], cls_dets.cpu().numpy(), 0.5)

    keep_boxes = torch.where(max_conf >= conf_thresh, max_conf, torch.tensor(0.0))
    keep_boxes = torch.squeeze(torch.nonzero(keep_boxes))
    if len(keep_boxes) < MIN_BOXES:
        keep_boxes = torch.argsort(max_conf, descending = True)[:MIN_BOXES]
    elif len(keep_boxes) > MAX_BOXES:
        keep_boxes = torch.argsort(max_conf, descending = True)[:MAX_BOXES]

    objects = torch.argmax(scores[keep_boxes][:,1:], dim=1)
    box_dets = np.zeros((len(keep_boxes), 4))
    boxes = pred_boxes[keep_boxes]
    obj_classes = []
    bboxes = []
    for i in range(len(keep_boxes)):
        bboxes.append(boxes[i])
        # bbox = (x_min,y_min,x_max,y_max)
        if bboxes[i][0] == 0:
           bboxes[i][0] = 1
        if bboxes[i][1] == 0:
           bboxes[i][1] = 1
        obj_classes.append(classes[objects[i]+1])
    return {
        'image_h': np.size(im, 0),
        'image_w': np.size(im, 1),
        'num_boxes': len(keep_boxes),
        'obj_names': obj_classes,
        'boxes': bboxes,
        'features': base64.b64encode((pooled_feat[keep_boxes].cpu()).detach().numpy())
    }

In [24]:
classes, model = load_model()
image_path = '../img/img2.jpg'
detected = get_detections_from_im(model,classes,image_path)

Using config:
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [4, 8, 16, 32],
 'CROP_RESIZE_WITH_MAX_POOL': False,
 'CUDA': False,
 'DATA_DIR': 'd:\\thesis\\data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'res101',
 'FEAT_STRIDE': [16],
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MAX_NUM_GT_BOXES': 20,
 'MOBILENET': {'DEPTH_MULTIPLIER': 1.0,
               'FIXED_LAYERS': 5,
               'REGU_DEPTH': False,
               'WEIGHT_DECAY': 4e-05},
 'PIXEL_MEANS': array([[[102.9801, 115.9465, 122.7717]]]),
 'POOLING_MODE': 'align',
 'POOLING_SIZE': 7,
 'RESNET': {'FIXED_BLOCKS': 1, 'MAX_POOL': False},
 'RNG_SEED': 3,
 'ROOT_DIR': 'd:\\thesis',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'MODE': 'nms',
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'gt',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000,
          'RPN_TOP_N': 5000,
          'SC

c:\Users\chobi\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


In [25]:
detected

{'image_h': 720,
 'image_w': 1440,
 'num_boxes': 15,
 'obj_names': ['woman',
  'woman',
  'rock',
  'sky',
  'man',
  'dog',
  'shirt',
  'shorts',
  'shirt',
  'dress',
  'man',
  'girl',
  'jeans',
  'leash',
  'building'],
 'boxes': [tensor([ 984.7740,  205.4018, 1117.4828,  548.2280]),
  tensor([4.5753e-01, 1.9268e+02, 7.2709e+01, 5.7990e+02]),
  tensor([1258.9001,  329.2609, 1438.0902,  521.5428]),
  tensor([2.8807e+02, 1.0000e+00, 1.4259e+03, 1.6756e+02]),
  tensor([767.9279, 117.8896, 950.1650, 639.5096]),
  tensor([ 872.6995,  445.1021, 1042.3125,  641.3333]),
  tensor([753.8677, 175.7476, 915.4963, 367.5348]),
  tensor([758.6577, 361.3621, 918.8499, 522.0151]),
  tensor([255.5024, 198.5121, 364.3859, 307.1043]),
  tensor([1145.4114,  362.3943, 1221.9432,  529.4482]),
  tensor([197.7707, 170.6035, 402.3687, 538.8360]),
  tensor([1095.7797,  295.3316, 1213.7833,  597.9448]),
  tensor([258.5963, 416.1529, 368.4823, 530.8174]),
  tensor([889.8365, 369.5089, 944.4954, 492.6424]),
 

In [26]:
def draw_bounding_box(img, obj_names, bboxes):

    for i, (name,box) in enumerate(zip(obj_names,bboxes)):
        
        x_min, y_min = int(box[0]), int(box[1])
        x_max, y_max = int(box[2]), int(box[3])
        color = (random.randint(0,255),random.randint(0,255),random.randint(0,255))
        cv2.rectangle(img,(x_min,y_min),(x_max,y_max), color, 0)
        cv2.putText(img, name, (x_min, y_min-5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (36,255,12), 0)
    cv2.imshow('man', img)
    cv2.waitKey(0)

In [28]:
img = cv2.imread('../img/img2.jpg')
names = detected.get('obj_names')
bboxes = detected.get('boxes')
draw_bounding_box(img,names,bboxes)
